# New Section

In [ ]:
from collections import defaultdict

def map_index(vertex, edges, current_distances):
    """Mapper function that processes vertices and their distances"""
    emitted_data = []

    # Emit current known distance
    emitted_data.append((vertex, (vertex, current_distances[vertex])))

    # Emit potential new distances through this vertex
    for neighbor, weight in edges.items():
        if current_distances[vertex] != float('inf'):
            new_distance = current_distances[vertex] + weight
            emitted_data.append((neighbor, (vertex, new_distance)))

    return emitted_data

def reduce_index(key, values):
    """Reducer function that finds minimum distances to each vertex"""
    min_distance = float('inf')
    source_vertex = None

    for src, dist in values:
        if dist < min_distance:
            min_distance = dist
            source_vertex = src

    return (key, (source_vertex, min_distance))


In [ ]:
def single_mapreduce_pass(graph, current_distances):
    """Performs a single MapReduce pass over the graph"""
    # MAP phase
    mapped_data = []
    for vertex, edges in graph.items():
        mapped_data.extend(map_index(vertex, edges, current_distances))

    # Group data by key for reduce phase
    grouped_data = defaultdict(list)
    for k, v in mapped_data:
        grouped_data[k].append(v)

    # REDUCE phase
    new_distances = {}
    for key, values in grouped_data.items():
        vertex, (source, distance) = reduce_index(key, values)
        new_distances[vertex] = distance

    return new_distances

In [ ]:
def iterative_mapreduce_dijkstra(graph, source, max_iterations=None):
    """Implementation of Dijkstra's algorithm using iterative MapReduce"""
    if max_iterations is None:
        max_iterations = len(graph) - 1  # Maximum path length in the graph

    # Initialize distances
    distances = {v: float('inf') for v in graph}
    distances[source] = 0

    # Keep track of distance changes between iterations
    iteration = 0
    changed = True

    while changed and iteration < max_iterations:
        old_distances = distances.copy()
        distances = single_mapreduce_pass(graph, distances)

        # Check if any distances changed in this iteration
        changed = any(old_distances[v] != distances[v] for v in distances)
        iteration += 1

        print(f"\nIteration {iteration}:")
        for vertex, dist in sorted(distances.items  ()):
            print(f"{vertex}: {dist}")

    return distances, iteration

In [ ]:
def main():
    # Example graph represented as adjacency list with weights
    graph = {
        'A': {'B': 4, 'C': 2},
        'B': {'A': 4, 'C': 1, 'D': 5},
        'C': {'A': 2, 'B': 1, 'D': 8, 'E': 10},
        'D': {'B': 5, 'C': 8, 'E': 2},
        'E': {'C': 10, 'D': 2}
    }

    source = 'A'
    shortest_paths, num_iterations = iterative_mapreduce_dijkstra(graph, source)

    print(f"\nFinal shortest paths from vertex {source} after {num_iterations} iterations:")
    for vertex, distance in sorted(shortest_paths.items()):
        print(f"{vertex}: {distance}")

if __name__ == "__main__":
    main()


Iteration 1:
A: 0
B: 4
C: 2
D: inf
E: inf

Iteration 2:
A: 0
B: 3
C: 2
D: 9
E: 12

Iteration 3:
A: 0
B: 3
C: 2
D: 8
E: 11

Iteration 4:
A: 0
B: 3
C: 2
D: 8
E: 10

Final shortest paths from vertex A after 4 iterations:
A: 0
B: 3
C: 2
D: 8
E: 10
